In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
import numpy as np
import tensorflow as tf

def load_and_resize_images_in_batches(base_dir, subfolders, batch_size=100, target_size=(224, 224), output_file=None):
    # Initialize lists or arrays to hold batches
    resized_images = []
    labels = []

    for subfolder in subfolders:
        folder_path = os.path.join(base_dir, subfolder)
        print(f"Loading images from: {folder_path}")
        label = subfolder

        image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i+batch_size]
            batch_images = []
            batch_labels = []
            
            for img_path in batch_files:
                try:
                    # Read and decode the image
                    img = tf.io.read_file(img_path)
                    img = tf.image.decode_image(img, channels=3)
                    
                    # Resize the image
                    img_resized = tf.image.resize(img, target_size)
                    
                    # Convert to NumPy array and add to batch
                    batch_images.append(np.array(img_resized))
                    batch_labels.append(label)
                    print(f"Processed {img_path}")
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
            
            # Convert batches to arrays and append to main list
            resized_images.extend(batch_images)
            labels.extend(batch_labels)

            # Optionally, save the batch to disk if output_file is provided
            if output_file:
                np.savez_compressed(output_file, X=np.array(resized_images), y=np.array(labels))
    
    return np.array(resized_images), np.array(labels)

# Define the base directory and subfolders
base_dir = r"D:\micro-doppler based target classification\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset"
subfolders = [
    r"3_long_blade_rotor\3_long_blades_rotor", 
    r"3_short_blade_rotor_1\3_short_blade_rotor_1", 
    r"Bird\Bird", 
    r"Bird+mini-helicopter_1\Bird+2_Blade_rotor_1", 
    r"drone_1\drone_1", 
    r"drone_2\drone_2", 
    r"RC plane_1\2_blade_rotor_1", 
    r"RC plane_2\2_blade_rotor_2"
]

# Load and resize images in batches
X, y = load_and_resize_images_in_batches(base_dir, subfolders, batch_size=100)

print(f"Loaded {len(X)} images with shape {X.shape}.")
print(f"Loaded {len(y)} labels.")

Loading images from: D:\micro-doppler based target classification\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset\3_long_blade_rotor\3_long_blades_rotor
Processed D:\micro-doppler based target classification\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset\3_long_blade_rotor\3_long_blades_rotor\figure1.jpg
Processed D:\micro-doppler based target classification\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset\3_long_blade_rotor\3_long_blades_rotor\figure10.jpg
Processed D:\micro-doppler based target classification\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset\3_long_blade_rotor\3_long_blades_rotor\figure100.jpg
Processed D:\micro-doppler based target classification\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset\3_long_blade_rotor\3_long_blades_rotor\figure101.jpg
Processed D:\micro-doppler based target classification\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset\3_long_blade_ro

In [6]:
X = np.array(X)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4049,) + inhomogeneous part.

In [4]:
import numpy as np
import tensorflow as tf

def resize_images_in_batches(images, batch_size=100, target_size=(224, 224)):
    resized_images = []
    
    for i in range(0, len(images), batch_size):
        batch = images[i:i+batch_size]
        
        # Resize each image in the batch
        batch_resized = [tf.image.resize(img, target_size).numpy().astype(np.float16) for img in batch]
        
        # Append resized batch to the final list
        resized_images.extend(batch_resized)
    
    return np.array(resized_images)

# Process in batches
X_resized = resize_images_in_batches(X, batch_size=100)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_resized, y, test_size=0.2, random_state=42)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#size of dataset
print(f"Training set size: {X_train.shape}, Training labels size: {y_train.shape}")
print(f"Validation set size: {X_val.shape}, Validation labels size: {y_val.shape}")
print(f"Testing set size: {X_test.shape}, Testing labels size: {y_test.shape}")

Training set size: (2591, 224, 224, 3), Training labels size: (2591,)
Validation set size: (648, 224, 224, 3), Validation labels size: (648,)
Testing set size: (810, 224, 224, 3), Testing labels size: (810,)


In [7]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

In [8]:
num_classes = len(np.unique(y))
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)
y_val_cat = to_categorical(y_val_encoded, num_classes)